<a href="https://colab.research.google.com/github/Pedroct06/Time-series/blob/main/TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sktime
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sktime.forecasting.naive import NaiveForecaster
from sktime.utils.plotting import plot_series

Recebo o dataset para analisar o mercado de ações americano

In [ ]:
file_path = "SP500.csv"
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "awadhi123/finance-data-sp-500",
  file_path,)
df.info()
print(df.head())

Transformo a minha coluna de data em dateTime, para poder colocar nos parâmetros do sktime

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

Transformo a coluna de data como sendo o índice do meu dataset

In [ ]:
df.set_index('Date', inplace=True)
df.head()

Declaro o Y, como sendo a coluna de fechamento(o que quero prever)

In [ ]:
y = df['Close']
y

Trato as linhas vazias, pois o mercado é fechado em fins de semana, então repito o valor contido nos dias anteriores

In [ ]:
y = y.asfreq('B').fillna(method='ffill')
y

Faço a decomposição da minha coluna de preços, para analisar a sasonalidade, se possui alguma tendencia de repetição, e se possui muitos resíduos

In [ ]:
!pip install statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(y.tail(500), model = 'additive', period = 5)

fig = result.plot()
fig.set_size_inches(12,8)
plt.show()

Separo os dados de treino e teste, dos últimos 30 dias serem para teste, e o restante ser de treino

In [ ]:
y_train = y.iloc[:-30]
y_test = y.iloc[-30:]

Utilizei o modelo de NaiveForecaster como sendo o primeiro, ele repete os últimos números

In [ ]:
modelo = NaiveForecaster(strategy = 'last')
modelo.fit(y_train)

In [ ]:
y_pred_naive = modelo.predict(fh = np.arange(1,31))

In [ ]:
plot_series(y_train.tail(100), y_test, y_pred_naive,labels = ['treino', 'real', 'predição'])

Prevendo agora com o modelo AutoARIMA, modelo que tenta achar automaticamente os melhores parâmetros para fazer uma boa previsão

In [ ]:
!pip install pmdarima

In [ ]:
!pip install pmdarima
from sktime.forecasting.arima import AutoARIMA

modelo = AutoARIMA(sp = 1)
modelo.fit(y_train.tail(500))

y_predArima = modelo.predict(fh = np.arange(1,31))

plot_series(y_train.tail(100), y_test, y_predArima,labels = ['treino', 'real', 'predição'])

Prevendo agora com o modelo AutoETS, tenta encontrar a melhor combinação para fazer uma suavização exponencial

---



In [ ]:
from sktime.forecasting.ets import AutoETS

modelo = AutoETS(auto = True, sp = 5)
modelo.fit(y_train.tail(500))

y_predAETS = modelo.predict(fh = np.arange(1,31))

plot_series(y_train.tail(100), y_test, y_predAETS,labels = ['treino', 'real', 'predição'])


Utilizando o modelo VAR, modelo multivariado que leva mais de uma variável em consideração, e todas são interdependentes

In [ ]:
from sktime.forecasting.var import VAR

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "awadhi123/finance-data-sp-500",
  file_path,)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

data = df[['Close', 'Volume']]
data = data.asfreq('B').ffill()
data_diff = data.diff().dropna()

treino_diff = data_diff.iloc[:-30]
test_diff = data_diff.iloc[-30:]

modelo = VAR(maxlags = 15)
modelo.fit(treino_diff)

y_pred_diff = modelo.predict(fh = np.arange(1,31))
y_pred = y_pred_diff.cumsum() + data.iloc[-1]

ultimo_preco = data['Close'].iloc[-(31)]
y_pred_VAR = ultimo_preco + y_pred_diff['Close'].cumsum()

y_treino_plot = data['Close'].iloc[-130:-30]
y_test_plot = data['Close'].iloc[-30:]

plot_series(y_treino_plot, y_test_plot, y_pred_VAR,labels = ['treino', 'real', 'predição'])

Utilizando o modelo PROPHET, funciona através de tentar achar a melhor curva

In [ ]:
from sktime.forecasting.fbprophet import Prophet

modelo = Prophet(daily_seasonality = False, weekly_seasonality = True, changepoint_prior_scale=0.5)

modelo.fit(y_train)

y_pred_Prophet = modelo.predict(fh = np.arange(1,31))

plot_series(y_train.tail(100), y_test, y_pred_Prophet,labels = ['treino', 'real', 'predição'])


Colocando as métricas de avaliação
MAPE: Representa a porcentagem de erro em relação da predição com a realidade
MAE: Erro médio em dólares, quanto menor, significa que errou menos
MSE: Erro quadrático padrão
MASE: Serve para avaliar se o modelo é bom ou não, preferindo sempre modelos em que a MASE é menor do que 1
Avaliando essas métricas, podemos observar que o modelo prophet se mostra superior a todos os outros

In [ ]:
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, mean_absolute_scaled_error, MeanSquaredError

modelos = {'Naive' : y_pred_naive, 'AutoARIMA' : y_predArima, 'AutoETS' : y_predAETS, 'VAR' : y_pred_VAR, 'Prophet' : y_pred_Prophet}
rmse = MeanSquaredError(square_root = True)

for nome, pred in modelos.items():
  print(nome)
  print('MAPE: ', mean_absolute_percentage_error(y_test, pred))
  print('MAE: ', mean_absolute_error(y_test, pred))
  print('RMSE: ', rmse(y_test, pred))
  print('MSE: ', mean_squared_error(y_test, pred))
  print('MASE: ', mean_absolute_scaled_error(y_test, pred, y_train = y_train))
  print()